In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import standings_api_calls

In [2]:
standings = standings_api_calls.main(league='all', csv=False)

NHL
MLB
NBA
NFL


In [4]:
standings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5982 entries, 0 to 31
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         5982 non-null   object 
 1   name         5982 non-null   object 
 2   percentage   5982 non-null   float64
 3   season_year  5982 non-null   int32  
 4   season       5982 non-null   object 
 5   league       5982 non-null   object 
dtypes: float64(1), int32(1), object(4)
memory usage: 303.8+ KB


In [28]:
standings[standings.season_year == 200]

,city,name,percentage,season_year,season,league,z_score
0,Los Angeles,Lakers,0.817,200,1999-00,NBA,1.968679
1,Indiana,Pacers,0.683,200,1999-00,NBA,1.136312
2,South Florida,Heat,0.634,200,1999-00,NBA,0.831939
3,Utah,Jazz,0.671,200,1999-00,NBA,1.061771
4,Portland,Trail Blazers,0.720,200,1999-00,NBA,1.366144
5,New York,Knicks,0.610,200,1999-00,NBA,0.682858
6,Phoenix,Suns,0.646,200,1999-00,NBA,0.906479
7,Philadelphia,76ers,0.598,200,1999-00,NBA,0.608318
8,San Antonio,Spurs,0.646,200,1999-00,NBA,0.906479
9,Charlotte,Hornets,0.598,200,1999-00,NBA,0.608318


In [10]:
standings['z_score'] = standings.groupby(['league', 'season_year'])['percentage'].transform(lambda x: (x - x.mean()) / x.std())

In [25]:
grouped_standings = standings.groupby(['season_year', 'city'])['z_score'].agg(
    ['sum', 'mean']
    ).reset_index()

In [26]:
grouped_standings

,season_year,city,sum,mean
0,200,Atlanta,-0.988088,-0.988088
1,200,Bay Area,-1.665162,-1.665162
2,200,Boston,-0.453882,-0.453882
3,200,Charlotte,0.608318,0.608318
4,200,Chicago,-1.820455,-1.820455
...,...,...,...,...
2726,2024,Utah,-0.745948,-0.745948
2727,2024,Vancouver,1.050756,1.050756
2728,2024,Vegas,0.427037,0.427037
2729,2024,Washington,-1.764401,-0.441100


In [23]:
grouped_standings.sort_values('z_score', ascending=False)

ValueError: The column label 'z_score' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

In [49]:
grouped_standings.to_csv('city_year_standings.csv')

In [8]:
standings[(standings.city == 'Chicago') & (standings['season_year'] == 2024)]

,city,name,percentage,season_year,season,league
1354,Chicago,Blackhawks,0.317073,2024,20232024,NHL
